In [14]:
from imsciences import *
import pandas as pd

ims = dataprocessing()

path = ims.get_wd_levels(0)
df = ims.read_and_concatenate_files(path, "csv")

In [ ]:
manual_dividends = [
    # Format: (Date, Ticker, Company Name, Total GBP, Shares, Price/Share, Currency, ISIN, Withholding Tax)
    # You can use None for unknown values
    ('2025-12-10', 'PYPL', 'PayPal Holdings Inc', 9.13, 103.0, 61.66, 'USD', '', 0.0),
]

# Process and add dividends
added_count = 0
skipped_count = 0

for entry in manual_dividends:
    date, ticker, name, total, shares, price, currency, isin, withholding = entry
    
    # Add time to date if not included
    time_str = f"{date} 12:00:00" if len(date) == 10 else date
    
    new_dividend = {
        'Action': 'Dividend (Dividend)',
        'Time': time_str,
        'ISIN': isin if isin else '',
        'Ticker': ticker,
        'Name': name,
        'No. of shares': shares if shares is not None else 0.0,
        'Price / share': price if price is not None else 0.0,
        'Currency (Price / share)': currency,
        'Exchange rate': 0.75,  # Default GBP/USD rate
        'Total': total,
        'Currency (Total)': 'GBP',
        'Withholding tax': withholding if withholding is not None else 0.0,
        'Currency (Withholding tax)': currency
    }
    
    # Check if dividend already exists
    existing = df[
        (df['Ticker'] == ticker) & 
        (df['Time'] == time_str) & 
        (df['Total'] == total)
    ]
    
    if existing.empty:
        df = pd.concat([df, pd.DataFrame([new_dividend])], ignore_index=True)
        print(f"✓ Added: {name} - £{total}")
        added_count += 1
    else:
        print(f"⚠ Skipped (exists): {name} - £{total}")
        skipped_count += 1

print(f"\nSummary: {added_count} added, {skipped_count} skipped, {len(df)} total rows")

✓ Added: PayPal Holdings Inc - £9.13

Summary: 1 added, 0 skipped, 450 total rows


In [13]:
filename = "dividends_up_to_dec12_isa_gia_2025_new.csv"
df.to_csv(filename, index=False)